In [ ]:
# notebooks/1_data_exploration.ipynb

import tensorflow as tf
import pyspark
from pyspark.sql import SparkSession
import cv2

# Configurar Spark
spark = SparkSession.builder \
    .appName("DocumentClassifier") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

def create_training_dataset():
    """
    Convert PDFs to images and organize training data
    """
    # Usar Spark para procesamiento paralelo de PDFs
    pdf_rdd = spark.sparkContext.wholeTextFiles("data/raw/*.pdf")
    
    # Convertir y preprocesar imágenes
    def process_pdf(pdf_file):
        images = convert_pdf_to_images(pdf_file)
        return [(img, detect_liquidacion_page(img)) for img in images]
    
    processed_images = pdf_rdd.flatMap(process_pdf)